In [1]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
#import plotly.express as px
import seaborn as sns

from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import StratifiedKFold
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, \
                            accuracy_score, f1_score, average_precision_score, \
                            fbeta_score, precision_recall_curve, auc
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import train_test_split

import optuna
from optuna.samplers import TPESampler
sns.set(style='white', context='notebook', palette='deep')
%config Completer.use_jedi = False

In [2]:
csl_df = pd.read_csv('../../data/csl/Sites/CSL_he_PI_s42.csv', index_col=0)
X = csl_df.drop(['high_EBL','Sitenum'], axis=1, inplace=False)
y = csl_df['high_EBL'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=7)
X_train                                                    

,transfus_yes,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,...,TD_nos,ThreatenedPB,threatpb9,UnspecHBP,uscar,version9,new_age,new_BMI,new_high_Age,new_high_BMI
MomID,,,,,,,,,,,,,,,,,,,,,
42-04780,0,0,8,8,8,99.0,77,100.0,1,121.0,...,0,0,0,0,0,0,35,25,1,1
42-02059,0,0,8,8,8,99.0,75,70.0,1,126.0,...,0,0,0,0,0,0,20,25,1,1
42-03840,0,0,8,8,8,99.0,74,100.0,1,121.0,...,0,0,0,0,0,0,21,26,1,1
42-00258,0,0,8,8,8,99.0,74,50.0,1,124.4,...,0,0,0,0,0,0,23,28,1,1
42-04735,0,0,8,8,8,99.0,79,100.0,1,151.0,...,1,0,0,0,0,0,20,38,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42-04313,0,0,8,8,8,99.0,81,90.0,1,115.0,...,0,0,0,0,0,0,17,24,0,1
42-05627,0,0,8,8,8,99.0,89,100.0,1,137.0,...,0,0,0,0,0,0,30,19,1,0
42-02719,0,0,8,8,8,99.0,76,100.0,1,127.0,...,0,0,0,0,0,0,24,24,1,1


In [3]:
X.shape

(5784, 195)

<h1 align='center'>Exploratory Data Analysis</h1>

<h1 align='center'>Modelling</h1>

In [8]:
print(X_train.shape)
print(X.shape)
print(y.shape)

(4048, 195)
(5784, 195)
(5784,)


Here we define our validation set

<h1 align='center'>Modelling Using Undersampling</h1>

We will use `imblearn`'s RandomUnderSampler to undersample from the majority class so that they match

In [9]:
sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
X_rus, y_rus = sampler.fit_resample(X_train, y_train)

# Try with no undersampling
#X_rus, y_rus = X_train, y_train

In [10]:
print(X_rus.shape)
print(y_rus.shape)
print(np.bincount(y_rus))

(398, 195)
(398,)
[199 199]


<h1 align='center'>Basic Gradient Boosting</h1>

In [11]:
%%time
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_rus = GradientBoostingClassifier()   # If not undersampling
gb_rus.fit(X_rus, y_rus)
print(gb_rus.get_params())
y_pred = gb_rus.predict(X_test)
probs = gb_rus.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[1294  356]
 [  12   74]]
              precision    recall  f1-score   support

           0       0.99      0.78      0.88      1650
           1       0.17      0.86      0.29        86

    accuracy                           0.79      1736
   macro avg       0.58      0.82      0.58      1736
weighted avg       0.95      0.79      0.85      1736

ROC AUC prob1: 0.8597392529950669
MCC: 0.3240769412945344
F2: 0.47803617571059437
PR_AUC: 0.23617949195964885
CPU times: user 216 ms, sys: 1.16 ms, tota

In [12]:
%%time
# Not undersampled
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_weight = GradientBoostingClassifier()  
weights = class_weight.compute_sample_weight('balanced', y=y_train)
gb_weight.fit(X_train, y_train, sample_weight=weights)
print(gb_weight.get_params())
y_pred = gb_weight.predict(X_test)
probs = gb_weight.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[1371  279]
 [  16   70]]
              precision    recall  f1-score   support

           0       0.99      0.83      0.90      1650
           1       0.20      0.81      0.32        86

    accuracy                           0.83      1736
   macro avg       0.59      0.82      0.61      1736
weighted avg       0.95      0.83      0.87      1736

ROC AUC prob1: 0.8586962649753348
MCC: 0.3491462884650192
F2: 0.5050505050505051
PR_AUC: 0.20716533440804316
CPU times: user 1.23 s, sys: 4.98 ms, total

<h1 align='center'>Hyperparameter Tuning with Cross Validation</h1>

In [13]:
%%time
def create_model(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    subsample = trial.suggest_float('subsample', 0.8, 1.0)
    max_depth = trial.suggest_int('max_depth',3,20)
    max_features = trial.suggest_int('max_features',10,20)
    learning_rate = trial.suggest_uniform('learning_rate',0.08,0.2)
    min_samples_leaf = trial.suggest_int('min_samples_leaf',1,50)
    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
                           max_depth=max_depth, learning_rate=learning_rate, 
                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
#                           learning_rate=learning_rate, 
#                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate) 
    return model

def objective(trial):
    model = create_model(trial)
    
    mcc_list = []
    roc_list = []
    pr_list = []
    f2_list = []
    cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=11)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train_c, X_test_c = X.iloc[train_idx], X.iloc[test_idx]
        y_train_c, y_test_c = y[train_idx], y[test_idx]
        sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
        X_res, y_res = sampler.fit_resample(X_train_c, y_train_c)

#        weights = class_weight.compute_sample_weight('balanced', y=y_train)
#        model.fit(X_train, y_train, sample_weight=weights)
        model.fit(X_res, y_res)
        y_pred = model.predict(X_test_c)
        probs = model.predict_proba(X_test_c)
        prob1 = probs[:, 1]  # Only positives
        precision, recall, pr_thresh = precision_recall_curve(y_test_c, prob1)
        roc = roc_auc_score(y_test_c, prob1)
        roc_list.append(roc)
        mcc = matthews_corrcoef(y_test_c, y_pred)  # MWB
        mcc_list.append(mcc)
        f2 = fbeta_score(y_test_c, y_pred, beta=2.0, average=None)[1]
        pr_auc = auc(recall, precision)
        pr_list.append(pr_auc)
        avg_precision = average_precision_score(y_test_c, prob1)
#        print(f'ROC: {roc}')
#        print(f'PR_AUC: {pr_auc}')
#        print(f'avg_prec: {avg_precision}')
        
    print(f'MCC: {mcc_list}')
    print(f'ROC: {roc_list}')
    print(f'PR_AUC: {pr_list}')
    return np.mean(mcc_list)

sampler = TPESampler(seed=7)
study = optuna.create_study(sampler=sampler,direction='maximize')
#study.optimize(objective,n_trials=60)
study.optimize(objective,n_trials=25)
#study.optimize(objective,n_trials=5)

[I 2021-12-10 17:12:01,051] A new study created in memory with name: no-name-8048c74c-fd79-4fe4-9aea-b76ae2dc9827
[I 2021-12-10 17:12:01,512] Trial 0 finished with value: 0.3449656297807747 and parameters: {'n_estimators': 24, 'subsample': 0.955983758448023, 'max_depth': 10, 'max_features': 17, 'learning_rate': 0.1973587414395923, 'min_samples_leaf': 27}. Best is trial 0 with value: 0.3449656297807747.


MCC: [0.3109672292879325, 0.35225932063795257, 0.3461078586784821, 0.3705281105187316]
ROC: [0.8495877080665812, 0.8798668373879641, 0.8690601792573623, 0.8889040514313439]
PR_AUC: [0.2542466127523184, 0.2650230299097522, 0.23002338928981045, 0.2507584793954582]


[I 2021-12-10 17:12:02,455] Trial 1 finished with value: 0.3376963176858768 and parameters: {'n_estimators': 105, 'subsample': 0.8144102266719524, 'max_depth': 7, 'max_features': 15, 'learning_rate': 0.16150759953451288, 'min_samples_leaf': 41}. Best is trial 0 with value: 0.3449656297807747.


MCC: [0.3190562309948463, 0.3485013459659645, 0.3281256479779031, 0.3551020458047932]
ROC: [0.8469551856594111, 0.8736389244558258, 0.8736696542893726, 0.8758187772925764]
PR_AUC: [0.23563687114886772, 0.2597903206089115, 0.23043793508677818, 0.260297260538613]


[I 2021-12-10 17:12:03,466] Trial 2 finished with value: 0.3392141740477944 and parameters: {'n_estimators': 82, 'subsample': 0.813187269381181, 'max_depth': 8, 'max_features': 20, 'learning_rate': 0.10560624242958987, 'min_samples_leaf': 23}. Best is trial 0 with value: 0.3449656297807747.


...

MCC: [0.32308071394446697, 0.3553684464547116, 0.3591524129233262, 0.36813405977226393]
ROC: [0.8547195902688861, 0.8924967989756722, 0.8691318822023048, 0.8911632702571567]
PR_AUC: [0.30458486768316834, 0.28794964853200034, 0.20271454104738915, 0.2678708626010756]


[I 2021-12-10 17:12:26,471] Trial 23 finished with value: 0.36416430137851385 and parameters: {'n_estimators': 25, 'subsample': 0.928784331939467, 'max_depth': 3, 'max_features': 13, 'learning_rate': 0.13938806574215398, 'min_samples_leaf': 34}. Best is trial 23 with value: 0.36416430137851385.


MCC: [0.34481634207253226, 0.3607194793435652, 0.35997241328162954, 0.39114897081632843]
ROC: [0.8502637644046095, 0.8952061459667093, 0.8726760563380281, 0.8932253760310528]
PR_AUC: [0.25538781759738954, 0.28023084505471535, 0.2333472032776976, 0.2655697190533627]


[I 2021-12-10 17:12:26,827] Trial 24 finished with value: 0.36054423817143805 and parameters: {'n_estimators': 15, 'subsample': 0.9676952127097791, 'max_depth': 4, 'max_features': 11, 'learning_rate': 0.16777006678480033, 'min_samples_leaf': 31}. Best is trial 23 with value: 0.36416430137851385.


MCC: [0.3401487451967217, 0.3568730478616745, 0.35997241328162954, 0.38518274634572647]
ROC: [0.8488450704225352, 0.8887426376440462, 0.8765275288092189, 0.887236171761281]
PR_AUC: [0.24871423451365715, 0.23918796732512776, 0.2488327671612125, 0.25770578226214796]
CPU times: user 25.7 s, sys: 287 ms, total: 25.9 s
Wall time: 25.8 s


In [14]:
best = study.best_trial
best

FrozenTrial(number=23, values=[0.36416430137851385], datetime_start=datetime.datetime(2021, 12, 10, 17, 12, 26, 26077), datetime_complete=datetime.datetime(2021, 12, 10, 17, 12, 26, 470392), params={'n_estimators': 25, 'subsample': 0.928784331939467, 'max_depth': 3, 'max_features': 13, 'learning_rate': 0.13938806574215398, 'min_samples_leaf': 34}, distributions={'n_estimators': IntUniformDistribution(high=200, low=10, step=1), 'subsample': UniformDistribution(high=1.0, low=0.8), 'max_depth': IntUniformDistribution(high=20, low=3, step=1), 'max_features': IntUniformDistribution(high=20, low=10, step=1), 'learning_rate': UniformDistribution(high=0.2, low=0.08), 'min_samples_leaf': IntUniformDistribution(high=50, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=23, state=TrialState.COMPLETE, value=None)

In [9]:
%%time
gb_params = study.best_params
#gb_params['random_state'] = 42
gb = GradientBoostingClassifier(**gb_params)
gb.fit(X_rus, y_rus)
y_pred = gb.predict(X_test)
probs = gb.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

NameError: name 'study' is not defined

In [16]:
%%time
gb_params = study.best_params
gb_params['random_state'] = 42
gb_weight = GradientBoostingClassifier(**gb_params)
weights = class_weight.compute_sample_weight('balanced', y=y_train)
gb_weight.fit(X_train, y_train, sample_weight=weights)
y_pred = gb_weight.predict(X_test)
probs = gb_weight.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[1310  340]
 [   8   78]]
              precision    recall  f1-score   support

           0       0.99      0.79      0.88      1650
           1       0.19      0.91      0.31        86

    accuracy                           0.80      1736
   macro avg       0.59      0.85      0.60      1736
weighted avg       0.95      0.80      0.85      1736

ROC AUC prob1: 0.8849999999999999
MCC: 0.3557225616740703
F2: 0.5118110236220472
PR_AUC: 0.2684872379620049
CPU times: user 96.9 ms, sys: 4.02 ms, total: 101 ms
Wall time: 98.9 ms


In [17]:
# No Sitenum - 5
coeffs_wgt = pd.Series(data=gb_weight.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs_wgt[0:20]

Delmode           0.270206
Lac_None          0.218308
Lac_Min           0.169190
CS_FTP            0.094985
ROMmeth           0.053427
CS_UScar          0.027081
vertex            0.018877
Antesteroid       0.018405
new_BMI           0.015608
CS_Elect          0.015287
prelaborCD        0.008262
intra_abruptio    0.007106
Admefface         0.006385
Admpresent        0.006126
BESTGA            0.005656
high_height       0.005648
IntraMgSO         0.005608
momrace_new       0.005472
high_Gravidity    0.004037
AdmDBP            0.003416
dtype: float64

In [12]:
plt.rcParams["figure.figsize"] = (14, 8)
x_labels = coeffs_wgt.index.values[0:25]
print(x_labels)
fig, ax = plt.subplots(1,1)  # Create a figure and an axes.
ax.set_title('Top Predictors: All Sites combined; target: High_EBLoss, vars: PI')
ax.bar(x_labels, coeffs_wgt.values[0:25])
ax.set_ylabel('Coeff')
ax.set_xlabel('Variable')
plt.draw()
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=16)
#plt.xlim(1,20)
plt.show()

NameError: name 'coeffs_wgt' is not defined

In [19]:
# No Sitenum - 5
coeffs = pd.Series(data=gb.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs[0:20]

Lac_None       0.254690
HxnumCS        0.136208
Lac_Min        0.111217
Delmode        0.090414
CS_FTP         0.083102
new_BMI        0.057152
Presentdel     0.039371
CS_Elect       0.034893
Dilat_lst      0.032201
Admpresent     0.027259
IUPC           0.024720
prelaborCD     0.017480
BESTGA         0.009879
IntraMgSO      0.009760
momrace_new    0.009092
Admreason      0.008747
Admefface      0.008615
Insurance      0.007905
Hxcsection     0.006252
Antesteroid    0.005026
dtype: float64

In [21]:
gb_params # 5 runs

{'n_estimators': 25,
 'subsample': 0.928784331939467,
 'max_depth': 3,
 'max_features': 13,
 'learning_rate': 0.13938806574215398,
 'min_samples_leaf': 34,
 'random_state': 42}

In [10]:
from optuna.visualization import plot_optimization_history

plotly_config = {"staticPlot": True}

fig = plot_optimization_history(study)
fig.show(config=plotly_config)

NameError: name 'study' is not defined

In [11]:
from optuna.visualization import plot_param_importances

fig = plot_param_importances(study)
fig.show(config=plotly_config)

NameError: name 'study' is not defined

In [24]:
np.mean([0.3149689279176914, 0.31449457458602254, 0.31066798538596907, 0.3219519143488398])

0.3155208505596307

<h1 align='center'>Thanks and make sure to learn!</h1>